In [ ]:
%matplotlib ipympl
import numpy as np
import tqdm
import matplotlib.pyplot as plt
from scipy import interpolate

from vizlib import *

In [ ]:
filename = "/Volumes/DATA/data_06_11_2023-5/amatter_06_11_2023-5_worms_only.xyzv"

N,nframes,pos_data,vel_data = load_data(filename)

In [ ]:
vector_field = get_particle_vecs_frame(N,-1,pos_data)

In [ ]:
"""plotting the vectors of each of the individual particles"""
fig,ax = plt.subplots(1,1)
quiver = ax.quiver(vector_field[:,0], #x
                   vector_field[:,1], #y
                   vector_field[:,3], #u
                   vector_field[:,4], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
ax.set_box_aspect(1)
ax.set_adjustable("datalim")
plt.show()

In [ ]:
vector_grid = interp_particle_vecs(vector_field)


In [ ]:
"""plotting the vectors of each grid"""
fig,ax = plt.subplots(1,1)
quiver = ax.quiver(vector_grid[:,0], #x
                   vector_grid[:,1], #y
                   vector_grid[:,2], #u
                   vector_grid[:,3], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
ax.set_box_aspect(1)
ax.set_adjustable("datalim")
plt.show()

In [ ]:
from matplotlib.animation import FuncAnimation

# Assuming you have a list of vector fields called 'vector_fields'
vector_fields = []
for iframe in tqdm.tqdm(range(nframes)):
    vector_field_frame = get_particle_vecs_frame(N,iframe,pos_data,disp=False)
    vector_grid_frame = interp_particle_vecs(vector_field_frame,disp=False)
    vector_fields.append(vector_grid_frame)
    
fig, ax = plt.subplots(1, 1)

def update(frame):
    ax.clear()
    vector_grid = vector_fields[frame]
    quiver = ax.quiver(vector_grid[:, 0], vector_grid[:, 1], vector_grid[:, 2], vector_grid[:, 3],
                       pivot='mid',
                       headlength=0,
                       headwidth=0,
                       headaxislength=0,
                       color="tab:blue",
                       scale_units='xy',
                       scale=0.25
                       )
    ax.set_box_aspect(1)
    ax.set_adjustable("datalim")
    ax.set_title(f'Timestep {frame}')

# Create the animation
animation = FuncAnimation(fig, update, frames=nframes, interval=200, repeat=False)
#saving the animation
animation.save('vector_field_animation.mp4', writer='ffmpeg', fps=10)
#plt.show()
